In [2]:
import cv2
import numpy as np
import os 
from datetime import datetime
import csv

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

#initiate id counter
id = 0
f = open("name.txt", 'r')
names=f.readlines()
f.close()


# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)

while True:

    #starting of camera 
    ret, img =cam.read()

    #Converting into gray scale 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 3,
        minSize = (int(minW), int(minH)),
       )

    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 3)

        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 90):
            names[id]=names[id].replace("\n", "")
            id = names[id]
            
            #Getting current date and time 
            now = datetime.now()
            current_date = now.strftime("%d/%m/%Y")
            current_time = now.strftime("%H:%M:%S")
            
            confidence = "  {0}%".format(round(100 - confidence))
            
            flag=False
            csvFile=[]
            tmp=current_date.split("/")[0]+ "_" + current_date.split("/")[1]+ "_" + current_date.split("/")[2]
            
            #Making of csv file with current date as name of file 
            try:
                u = open('Attendance/'+tmp + ".csv")
                u.close()
            except FileNotFoundError as e:
                u=open('Attendance/'+tmp + ".csv", mode ='w+')
                u.close()
                
            #Checking if employee already added to the attendance sheet 
            with open('Attendance/'+tmp + ".csv", mode ='r+')as file:
                csvFile = file.readlines()
            for row in csvFile:
                if(row.split(",")[0]==id):
                    flag=True
                    
            f = open('Attendance/'+ tmp + ".csv",'a+',newline = '')
            lnwriter = csv.writer(f)
            
            #If employee not present adding to csv file 
            if(flag==False):
                print(id +  " Added")
                lnwriter.writerow([id,current_date,current_time])
            f.close()
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    
    cv2.imshow('camera',img) 

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 [INFO] Exiting Program and cleanup stuff
